In [1]:
import numpy as np
from scipy.spatial import distance

In [2]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    @property
    def as_np(self):
        return np.array([self.y, self.x])

def parse_point(s):
    x, y = s.split(',')
    return Point(int(x), int(y))

In [10]:
max_x = -np.inf
max_y = -np.inf
points = {}
with open("input.txt", "r") as fp:
    for l in fp.readlines():
        key = l.strip()
        p = parse_point(key)
        points[key] = p
        if p.x > max_x:
            max_x = p.x
        if p.y > max_y:
            max_y = p.y
p_list = list(points.keys())

In [11]:
masks = []
ogrid = np.ogrid[0:max_y+1, 0:max_x+1]
for k in points:
    masks.append(distance.cityblock(points[k].as_np, ogrid))
masks = np.array(masks)

In [12]:
group_mask = np.argmin(masks, axis=0)
group_mask

array([[14, 14, 14, ..., 45, 45, 45],
       [14, 14, 14, ..., 45, 45, 45],
       [14, 14, 14, ..., 45, 45, 45],
       ...,
       [33, 33, 33, ..., 34, 34, 34],
       [33, 33, 33, ..., 34, 34, 34],
       [33, 33, 33, ..., 34, 34, 34]])

In [13]:
dup_mask = np.ones(group_mask.shape)
z, y, x = masks.shape
for i in range(y):
    for j in range(x):
        arr = masks[:, i, j]
        if len(arr[arr == arr.min()]) > 1:
            dup_mask[i, j] = np.nan
dup_mask

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [14]:
region_mask = group_mask * dup_mask
region_mask

array([[14., 14., 14., ..., 45., 45., 45.],
       [14., 14., 14., ..., 45., 45., 45.],
       [14., 14., 14., ..., 45., 45., 45.],
       ...,
       [33., 33., 33., ..., 34., 34., 34.],
       [33., 33., 33., ..., 34., 34., 34.],
       [33., 33., 33., ..., 34., 34., 34.]])

In [15]:
boundary = np.ones_like(region_mask)
boundary[1:-1, 1:-1] = np.nan
boundary_mask = boundary * region_mask
boundary_mask

array([[14., 14., 14., ..., 45., 45., 45.],
       [14., nan, nan, ..., nan, nan, 45.],
       [14., nan, nan, ..., nan, nan, 45.],
       ...,
       [33., nan, nan, ..., nan, nan, 34.],
       [33., nan, nan, ..., nan, nan, 34.],
       [33., 33., 33., ..., 34., 34., 34.]])

In [16]:
max_area = 0
for i, p in enumerate(p_list):
    if i in boundary_mask:
        continue
    else:
        area = len(region_mask[region_mask == i])
        if area > max_area:
            max_area = area
            max_p = p
print(max_area)
print(max_p)

3840
131, 147
